In [1]:
# Imports
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup 
#from tabulate import tabulate

In [2]:
#extract the data from the website
url = requests.get("https://www.fundsexplorer.com.br/ranking")

In [3]:
fii_bs4 = BeautifulSoup(url.text, "html.parser")

In [4]:
#find the table with a tag
fii_table = fii_bs4.findAll(attrs = {'id' : 'scroll-wrapper'})
table = fii_table[0].findAll('table')

In [5]:
fii_df = pd.read_html(str(fii_table[0]))[0]

In [ ]:
#initial dataframe
fii_df

In [ ]:
fii_df.columns.values

In [8]:
#rename the columns
fii_df.columns = ['ticker', 'Setor', 'price', 'liquidez',
       'Dividendo', 'DividendYield', 'DY (3M)Acumulado',
       'DY (6M)Acumulado', 'dy', 'DY (3M)Média',
       'DY (6M)Média', 'Dividenmediio', 'DY Ano', 'Variação Preço',
       'Rentab.Período', 'Rentab.Acumulada', 'PatrimônioLíq.', 'VPA',
       'pvpa', 'DYPatrimonial', 'VariaçãoPatrimonial',
       'Rentab. Patr.no Período', 'Rentab. Patr.Acumulada',
       'VacânciaFísica', 'VacânciaFinanceira', 'QuantidadeAtivos']

In [ ]:
fii_df.columns.values

In [10]:
#save a backup od the data
fii_back = fii_df.copy()

In [11]:
#select the columns needed for the test
test = fii_df[['ticker', 'price', 'dy', 'pvpa','liquidez']]

In [ ]:
print("--> DATA:")
display(test.isnull().sum())

In [13]:
#excludes the "nan" data
test = test[test['price'].notna()]
test = test[test['dy'].notna()]

In [ ]:
print("--> DATA:")
display(test.isnull().sum())

In [15]:
#classified the dataframe by dividend yeld from largest to smallest 
test = test.sort_values(['dy'], ascending=False)

In [ ]:
test

In [17]:
# excludes the "R$" unit from the data
unit = 'R$'
test['price'] = [sub.replace(unit, "").strip() for sub in test['price']]

In [ ]:
test

In [ ]:
#view the types of the columns
test.dtypes

In [ ]:
#change the separator digit of "," for "."
test['price'] = test['price'].str.replace(',', '.')
test['price'] = pd.to_numeric(test['price'], errors='coerce')
test

In [21]:
#excludes the "nan" data
test = test[test['price'].notna()]

In [ ]:
#calculate the daily liquidity
test['lid2'] = test['price'] * test['liquidez']
test

In [23]:
##classified the dataframe by daily liquidity from largest to smallest 
test = test.sort_values(['lid2'], ascending=False)

In [24]:
#reset the index
test.reset_index(inplace=True, drop=True)

In [ ]:
#excludes the reits (fii) with daily liquidity less than R$ 200,000.00
indexNames = test[ (test['lid2'] <= 200000)].index
test.drop(indexNames , inplace=True)
test

In [26]:
#save the number of lines
ts = test.shape
ts = ts[0]

In [27]:
#classified the dataframe by dividend yeld from largest to smallest
test = test.sort_values(['dy'], ascending=False)
test.reset_index(inplace=True, drop=True)

In [28]:
b1 = [*range(0, ts, 1)]

In [29]:
#creates a ranking from highest to lowest dividend
test['b1'] = pd.Series(b1)

In [ ]:
test

In [31]:
#classified the dataframe by pvpa from largest to smallest
test = test.sort_values(['pvpa'], ascending=True)

In [32]:
test.reset_index(inplace=True, drop=True)

In [33]:
#creates a ranking from highest to lowest pvpa
b2 = [*range(0, ts, 1)]
test['b2'] = pd.Series(b2)

In [ ]:
test

In [35]:
#sum the dividend yeld rank with the pvpa rank
test = test.eval('Rank = b1 + b2')

In [36]:
#classified the dataframe by final rank from smallest to largest
test = test.sort_values(['Rank'], ascending=True)

In [37]:
test.reset_index(inplace=True, drop=True)

In [67]:
#select the top 10 reits (fii) to invest
indexNames = test[(test.index > 9)].index
test.drop(indexNames , inplace=True)

In [ ]:
y = [10,10,10,10,10,10,10,10,10,10]
ml = test['ticker']
me = [0.2,0,0,0,0,0,0,0,0,0]
plt.pie(y, labels = ml, shadow=True, autopct='%1.1f%%', explode=me)
plt.title('REITS RANK',fontname="Arial",fontweight="bold", size=20)
plt.legend()